## Data preparation

Before the data can be sent through the KNN model it needs to be tidied. 
Here I will: 

* Remove nulls;
* Create dummies as required;
* Normalise values to prevent exponential scales;
* Further reduce the data set to only relevant classifiers. 



In [1]:
# Import libraries

import pandas as pd

from sqlalchemy import create_engine

In [4]:
# Create connections to data and read data to pandas dataframe
engine = create_engine('sqlite:///../data/customers_with_behaviours.db')
df_initial = pd.read_sql_table('customers_with_behaviours', engine)
df_initial.head()

,UNITID,INSTNM,IALIAS,CITY,STABBR,FIPS,OBEREG,GENTELE,EIN,DUNS,...,AMOUNT_OF_INTERACTIONS_W_SALES_RNG,AMOUNT_OF_CALLS,AMOUNT_OF_CALLS_RNG,AMOUNT_OF_MESSAGES,AMOUNT_OF_MESSAGES_RNG,ENGAGED_WITH_MESSAGING,REACHED_NOT_ENGAGED_WITH_MESSAGING,ATTENDED_WEBINARS,WEBINAR_ATTENDANCE_SIZE,WEBINAR_ATTENDANCE_SIZE_RNG
0,100654,Alabama A & M University,AAMU,Normal,AL,1,5,2563725000,636001109,197216455,...,[51 - 100],18,[1 - 50],36,[1 - 50],1,1,1,15,[11 - 15]
1,100663,University of Alabama at Birmingham,,Birmingham,AL,1,5,2059344011,636005396,063690705,...,[301 - 400],292,[201 - 300],37,[1 - 50],0,1,0,0,[0]
2,100690,Amridge University,Southern Christian University Regions University,Montgomery,AL,1,5,33438738777550,237034324,126307792,...,[401 - 500],66,[51 - 100],371,[301 - 400],1,1,1,17,[16 - 20]
3,100706,University of Alabama in Huntsville,UAH University of Alabama Huntsville,Huntsville,AL,1,5,2568246120,630520830,949687123,...,[101 - 200],117,[101 - 200],74,[51 - 100],1,1,1,20,[16 - 20]
4,100724,Alabama State University,,Montgomery,AL,1,5,3342294100,636001101,040672685,...,[101 - 200],11,[1 - 50],151,[101 - 200],0,0,0,0,[0]
